In [2]:
import os,json
import random
import shutil
from tqdm import tqdm
split_dataset_flag = True # 运行之前先确认下是否真的需要划分数据集

In [3]:
# random.seed(101001)
random.random()

0.9307414910958935

## 将数据集 划分为 训练集/验证集

In [4]:
# 该代码段不一定要之执行
tot_corpus = r'det_bookspine_text/new_label_samples'

# ---------- 为训练集准备数据 --------
train_img_dir = r'det_bookspine_text/train'
if os.path.exists(train_img_dir):
    shutil.rmtree(train_img_dir)
os.makedirs(train_img_dir)

# ---------- 为验证集准备数据 --------
val_img_dir = r'det_bookspine_text/val'
if os.path.exists(val_img_dir):
    shutil.rmtree(val_img_dir)
os.makedirs(val_img_dir)


train_ratio = 0.85
if split_dataset_flag:
    for i in tqdm(os.listdir(tot_corpus)):
        if i.endswith('.jpg'):
            img_path = os.path.join(tot_corpus, i)
            json_path = os.path.join(tot_corpus, i.replace('.jpg', '.json'))
            if random.random() < train_ratio:
                # 这里面都为训练数据
                dst_img_path = os.path.join(train_img_dir, i)
            else: # 这里面都为验证数据
                dst_img_path = os.path.join(val_img_dir, i)
            shutil.copy(img_path, dst_img_path)

100%|██████████| 5830/5830 [00:02<00:00, 2290.30it/s]


# paddleOCR-det 数据格式要求 https://github.com/PaddlePaddle/PaddleOCR/blob/main/doc/doc_ch/dataset/ocr_datasets.md
" 图像文件名        中间用"\t"分隔      json.dumps编码的图像标注信息"
ch4_test_images/img_61.jpg    [{"transcription": "MASA", "points": [[310, 104], [416, 141], [418, 216], [312, 179]]}, {...}]

## 读取标签文件（.json） 中的内容

In [5]:
def get_corrd_from_json(json_path):
    with open(json_path, 'r', encoding='utf-8') as f:
        json_data = json.load(f)['shapes']
    # print(f"json_data:{json_data}")
    # contous_list = []    
    boxes_info_list = [] 
    for zone_info in json_data:
        # print(f"zone_info:{zone_info}")
        points = zone_info["points"]
        classes = zone_info["label"]
        # points = [tuple(point) for point in points]
        boxes_info_list.append({"box":points, "class":classes})
    return boxes_info_list

In [6]:
# test/1123.jpg	 [ 
#                 {"transcription": 0, "points": [[202, 129], [221, 124], [241, 120], [261, 116], [280, 111], [300, 107], [320, 103], [319, 83], [299, 87], [279, 92], [259, 97], [239, 101], [219, 106], [200, 111]]}, 
#                  {"transcription": 0, "points": [[183, 174], [210, 167], [238, 161], [266, 155], [294, 149], [322, 143], [350, 137], [347, 105], [318, 111], [289, 117], [260, 124], [231, 130], [202, 136], [174, 143]]}, 
#                  {"transcription": 0, "points": [[208, 202], [228, 199], [249, 196], [269, 193], [290, 190], [310, 187], [331, 185], [328, 144], [306, 149], [284, 154], [262, 159], [240, 164], [218, 169], [196, 175]]}, 
#                  {"transcription": 0, "points": [[1, 319], [9, 318], [18, 317], [27, 316], [36, 315], [45, 314], [54, 313], [47, 275], [39, 276], [31, 278], [24, 279], [16, 281], [8, 282], [1, 284]]}, 
#                  {"transcription": 0, "points": [[9, 333], [27, 332], [46, 332], [65, 332], [83, 331], [102, 331], [121, 331], [117, 307], [97, 310], [78, 314], [59, 318], [39, 321], [20, 325], [1, 329]]}, 
#                  {"transcription": 0, "points": [[474, 273], [477, 272], [481, 272], [485, 272], [489, 272], [493, 272], [497, 272], [497, 236], [491, 237], [485, 238], [479, 239], [473, 240], [467, 241], [461, 242]]}, 
#                  {"transcription": 0, "points": [[216, 248], [228, 235], [244, 227], [259, 220], [275, 214], [292, 210], [310, 209], [311, 185], [291, 188], [272, 194], [253, 199], [236, 210], [219, 219], [203, 230]]}
#                 ]


In [7]:
import os,cv2,json
import numpy as np
from rapidocr_onnxruntime import RapidOCR
engine = RapidOCR(det_model_path=None, cls_use_cuda=True, rec_use_cuda=True)






2025-04-14 16:18:12,856 - OrtInferSession - WARNING: CUDAExecutionProvider is not in available providers (['AzureExecutionProvider', 'CPUExecutionProvider']). Use AzureExecutionProvider inference by default.
2025-04-14 16:18:12,858 - OrtInferSession - INFO: !!!Recommend to use rapidocr_paddle for inference on GPU.
2025-04-14 16:18:12,859 - OrtInferSession - INFO: (For reference only) If you want to use GPU acceleration, you must do:
2025-04-14 16:18:12,860 - OrtInferSession - INFO: First, uninstall all onnxruntime pakcages in current environment.
2025-04-14 16:18:12,862 - OrtInferSession - INFO: Second, install onnxruntime-gpu by `pip install onnxruntime-gpu`.
2025-04-14 16:18:12,863 - OrtInferSession - INFO: 	Note the onnxruntime-gpu version must match your cuda and cudnn version.
2025-04-14 16:18:12,864 - OrtInferSession - INFO: 	You can refer this link: https://onnxruntime.ai/docs/execution-providers/CUDA-EP.html
2025-04-14 16:18:12,866 - OrtInferSession - INFO: Third, ensure CUDAEx

In [8]:
def make_label(tot_corpus = r"det_bookspine_text/new_label_samples",
               img_dir = r'det_bookspine_text/train',
               datatype = 'train' # 'test'
               ):
    
    
    with open('det_bookspine_text/%s.txt'%datatype, 'w', encoding='utf-8') as write:
        for i in os.listdir(img_dir):
            if i.endswith('.jpg'):
                img_path = os.path.join(tot_corpus, i)
                cv_img = cv2.imread(img_path)
                
                json_path = os.path.join(tot_corpus, i.replace('.jpg','.json'))
                contous_list = get_corrd_from_json(json_path) # contous_list = [ {}]
                # print(f'-----------------从json标签文件中获取的内容：{contous_list}')
                
                # engine.get_crop_img_list(cv_img,[ np.array(i) for i in contous_list])

                
                
                # print(f'从json标签文件中获取的内容：{contous_list}')
                boxes =[ [ [int(x), int(y) ] for x, y in contous["box"] ] for contous in  contous_list]
                # print("="*8)
                det_boxes_list = [np.array(box,dtype=np.float32) for box in boxes]
                ########### 由于是人工标注的box，可能会出现多边形的情况，需要将多边形转换为四边形 #############
                det_boxes_list_new = []
                for box in det_boxes_list:
                    if box.shape[0] == 4:
                        det_boxes_list_new.append(box)
                    else:
                        box = cv2.minAreaRect(box)
                        box = cv2.boxPoints(box)
                        box = np.array(box, dtype=np.float32)
                        det_boxes_list_new.append(box)
                
                #########################################################################################
                crop_img_list = engine.get_crop_img_list(cv_img,det_boxes_list_new)
                # print(f"---------------------rec_list:{crop_img_list}")
                res_text_list = []
                for crop_img in crop_img_list:
                    result = engine(crop_img,use_det=False)
                    res_text_list.append(result[0][0][0])
                
                classes = [contous["class"] for contous in  contous_list]
                this_img_det_rec = []
                for box_, class_, text in zip(boxes, classes, res_text_list):
                    # contous 是一个文本框
                    # [[x0,y0],[x1,y1],[x2,y2],[x3,y3]]
                    print(f'识别的文字：{text}')
                    contous_con = {
                        "transcription": text,   # TODO: 将 'text' ===> 文本框的文本内容
                        "points": box_,
                        "class":  "other-1" if class_ == 'other' else class_
                    }
                    this_img_det_rec.append(contous_con)
                if len(this_img_det_rec) == 0:
                    continue
                want_write_con = datatype + "/" + i + '\t' + json.dumps(this_img_det_rec,  ensure_ascii=False) + '\n'
                write.write(want_write_con)   
                
make_label()        
make_label(img_dir=r'det_bookspine_text/val',datatype='val') 
                
            

识别的文字：高等院校人力资源管理专业一十二五一规划系列教材
识别的文字：人员素质测评理论与方法
识别的文字：C962
识别的文字：2217
识别的文字：上海图书馆
识别的文字：钟学技木出燃社
识别的文字：伟大导师
识别的文字：马克思生平故事
识别的文字：李朋高德宝
识别的文字：出版社
识别的文字：上海图书馆
识别的文字：4472
识别的文字：A711
识别的文字：中国易学史略
识别的文字：得海燕落
识别的文字：版社
识别的文字：CSS DONA]
识别的文字：2234
识别的文字：B221.5
识别的文字：1124-12
识别的文字：赢在气度
识别的文字：出版社
识别的文字：王巍
识别的文字：雕在格局
识别的文字：B848.4-49
识别的文字：上海服馆
识别的文字：选贤用才
识别的文字：美福
识别的文字：人学服加酒院
识别的文字：出H
识别的文字：中国人才究会编
识别的文字：齐务生著
识别的文字：社
识别的文字：上海图书馆
识别的文字：破解人格基因
识别的文字：滋
识别的文字：第2
识别的文字：【美一高尔·巴伦糖茶·著
识别的文字：极简个性心理学
识别的文字：B848/7722-1
识别的文字：：65
识别的文字：上海图书馆
识别的文字：是这样练成的
识别的文字：超强大脑
识别的文字：B80
识别的文字：1233
识别的文字：团筑
识别的文字：鬼谷子
识别的文字：6
识别的文字：一春核成国一鬼谷子著
识别的文字：B228-64
识别的文字：2581-4#6
识别的文字：时数
识别的文字：上海图书馆
识别的文字：对个人主义的反思
识别的文字：钱满素著
识别的文字：修订本
识别的文字：爱默生和中国
识别的文字：B712.41
识别的文字：8735-1
识别的文字：社
识别的文字：1424-6
识别的文字：与师父有约
识别的文字：中出版社
识别的文字：于佩桦○著
识别的文字：疗愈的力量
识别的文字：B848.4-49
识别的文字：上海日书馆
识别的文字：0437-2
识别的文字：刘永刚/著
识别的文字：你可以成为更好的自己
识别的文字：孩子，
识别的文字：上
识别的文字：B848.4-49
识别的文字：JEESS
识别的文字：齐成最强大脑
识别的文字：2722
识别的文字：版社
识别的文字：逻辑脑
识别的文字：不用妈妈教，孩子自学
识别的文字：[日]山崎红
识别的文字：

In [9]:
[[[274.0, 94.0], [601.0, 94.0], [601.0, 145.0], [274.0, 145.0]], [[1817.0, 53.0], [2021.0, 53.0], [2021.0, 97.0], [1817.0, 97.0]]]

[[[274.0, 94.0], [601.0, 94.0], [601.0, 145.0], [274.0, 145.0]],
 [[1817.0, 53.0], [2021.0, 53.0], [2021.0, 97.0], [1817.0, 97.0]]]